## Load data into Snowflake tables
To begin, download the full dataset (1 million rows) as a zip from this [Kaggle link](https://www.kaggle.com/datasets/sridharstreaks/insurance-data-for-machine-learning). Then unzip it to a .csv. Load that .csv into your notebook's files directory. We will pull from that csv to create a training data table and our incoming "streamed" data table.

In [ ]:
# Import python packages
import pandas as pd

# Create the session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


Load data from csv into Pandas DataFrame

In [ ]:
# Load full 1M dataset into dataframe
insurance_df = pd.read_csv('insurance_dataset.csv')

Data cleaning, rearranging columns

In [ ]:
# Capitalize column names
insurance_df.columns = insurance_df.columns.str.upper()

# Rearrange columns to fit target schema
cols = insurance_df.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
insurance_df = insurance_df[cols]

Use the write_pandas() method to write the first 10k rows into the 'SOURCE_OF_TRUTH' table created with the SQL commands in the SQL file. The method "returns a Snowpark DataFrame object referring to the table where the pandas DataFrame was written to." (Snowpark Documentation)

In [ ]:
source_of_truth_df = session.write_pandas(insurance_df[:10000], table_name='SOURCE_OF_TRUTH',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)

The code below writes the remaining 990k to the INCOMING_DATA_SOURCE table to simulate data being streamed in

In [ ]:
incoming_data_source_df = session.write_pandas(insurance_df[10000:], table_name='INCOMING_DATA_SOURCE',database='INSURANCE',schema='ML_PIPE',auto_create_table=True)